# ALGORITMOS BASADOS EN SERIES TEMPORALES

## 1) Modelo de Promedio Ponderado de Ventas de Periodos Anteriores (PPPA)
* Toma como base de cálculo el Período Actual de la Ventana de Reposición Definida.
* El mismo período del mes anterior.
* El mismo período del año anterior.
* Con los 3 valores hace un promedio simple ponderado
* Podría agregarse mayor peso a cada uno de los factores si hiciera falta.

### Estee modelo calcula la demanda como un promedio ponderado de:

* **Mes en curso (t)** → Refleja la demanda actual.
* **Mes anterior (t-1)** → Captura tendencias recientes.
* **Mismo mes del año anterior (t-12)** → Captura estacionalidad.

### Fórmula General:


\begin{equation}
𝐷𝑡=𝛼⋅𝑉𝑡 + 𝛽⋅𝑉𝑡−1 + 𝛾⋅𝑉𝑡-12

\end{equation}

#### Donde:
* Dt = Demanda estimada del mes actual.
* 𝑉𝑡 = Ventas del mes en curso.
* 𝑉𝑡−1 = Ventas del mes anterior.
* 𝑉𝑡−12 = Ventas del mismo mes del año anterior.
* 𝛼,𝛽,𝛾 = Pesos asignados a cada variable, definidos con base en la importancia de cada factor.

### 📌 Este modelo es útil cuando la demanda sigue un patrón estacional anual y presenta tendencias recientes que afectan las ventas.


### 1) RUTINA PRINCIPAL

In [ ]:
# Importar Librerias
import pandas as pd
import numpy as np
from datetime import datetime
# Mostrar los resultados
import ace_tools_open as tools

# Evitar Mensajes Molestos
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category= FutureWarning)

In [14]:
# FUNCIONES

def forecast_basic(df, current_date, period_length=30):
    """
    Calcula la demanda estimada utilizando el algoritmo básico.

    Parámetros:
    - df: DataFrame de pandas con la información histórica de ventas.
    - current_date: Fecha de referencia para el cálculo.
    - period_length: Número de días que conforman el período (por defecto 30).

    El algoritmo utiliza tres períodos:
      1. Últimos 'period_length' días: desde current_date - (period_length - 1) hasta current_date.
      2. Los 'period_length' días anteriores: desde current_date - (2*period_length - 1) hasta current_date - period_length.
      3. El mismo período del año anterior: desde current_date - 1 año - (period_length - 1) hasta current_date - 1 año.

    La demanda estimada se calcula como el promedio de las ventas en estos tres períodos.
    """
    # Facctores de Pnderación
    # de Cada uno de los 3 períodos
   
    factor_last = 70
    factor_previous = 20
    factor_year = 10


    # Definir rangos de fechas para cada período
    last_period_start = current_date - pd.Timedelta(days=period_length - 1)
    last_period_end = current_date

    previous_period_start = current_date - pd.Timedelta(days=2 * period_length - 1)
    previous_period_end = current_date - pd.Timedelta(days=period_length)

    same_period_last_year_start = current_date - pd.DateOffset(years=1) - pd.Timedelta(days=period_length - 1)
    same_period_last_year_end = current_date - pd.DateOffset(years=1)

    # Filtrar los datos para cada uno de los períodos
    df_last = df[(df['Fecha'] >= last_period_start) & (df['Fecha'] <= last_period_end)]
    df_previous = df[(df['Fecha'] >= previous_period_start) & (df['Fecha'] <= previous_period_end)]
    df_same_year = df[(df['Fecha'] >= same_period_last_year_start) & (df['Fecha'] <= same_period_last_year_end)]
 
    # Agregar las ventas (unidades) por artículo y sucursal para cada período
    sales_last = df_last.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                        .sum().reset_index().rename(columns={'Unidades': 'ventas_last'})
    sales_previous = df_previous.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                                .sum().reset_index().rename(columns={'Unidades': 'ventas_previous'})
    sales_same_year = df_same_year.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                                  .sum().reset_index().rename(columns={'Unidades': 'ventas_same_year'})

    # Unir la información de los tres períodos
    forecast_df = pd.merge(sales_last, sales_previous, on=['Codigo_Articulo', 'Sucursal'], how='outer')
    forecast_df = pd.merge(forecast_df, sales_same_year, on=['Codigo_Articulo', 'Sucursal'], how='outer')
    forecast_df.fillna(0, inplace=True)

    # Calcular la demanda estimada como el promedio de las ventas de los tres períodos
    forecast_df['forecast'] = (forecast_df['ventas_last'] * factor_last +
                               forecast_df['ventas_previous'] * factor_previous +
                               forecast_df['ventas_same_year'] * factor_year) / (factor_year + factor_last + factor_previous)

    # Redondear la predicción al entero más cercano
    forecast_df['forecast'] = forecast_df['forecast'].round().astype(int)

    return forecast_df

def evaluate_forecast(forecast_df, actual_df, merge_keys=['Codigo_Articulo', 'Sucursal']):
    """
    Evalúa la precisión del algoritmo de predicción calculando métricas de error.

    Parámetros:
    - forecast_df: DataFrame con las predicciones (debe incluir la columna 'forecast').
    - actual_df: DataFrame con las ventas reales (debe incluir la columna 'actual').
    - merge_keys: Lista de columnas en las que se realizará la unión de ambos DataFrames.

    Retorna un diccionario con las métricas MAE, MSE y MAPE.
    """
    # Unir las predicciones con los valores reales
    merged = pd.merge(forecast_df, actual_df, on=merge_keys, how='inner')

    # Calcular las métricas de error
    mae = np.mean(np.abs(merged['forecast'] - merged['actual']))
    mse = np.mean((merged['forecast'] - merged['actual'])**2)
    # Se añade un valor pequeño para evitar división por cero
    mape = np.mean(np.abs((merged['forecast'] - merged['actual']) / (merged['actual'] + 1e-9))) * 100

    return {'MAE': mae, 'MSE': mse, 'MAPE': mape}

def get_forecast(df, algorithm='basic', current_date=None, period_length=30):
    """
    Permite la selección del algoritmo de predicción y calcula la demanda estimada.
    En el futuro podrémos ir agregando algoritmos

    Parámetros:
    - df: DataFrame con la información histórica.
    - algorithm: Algoritmo a utilizar (por defecto 'basic').
    - current_date: Fecha de referencia para el cálculo; si es None, se toma la fecha máxima del DataFrame.
    - period_length: Número de días del período a analizar.

    Retorna un DataFrame con las predicciones.
    """
    if current_date is None:
        current_date = df['Fecha'].max()
    else:
        current_date = pd.to_datetime(current_date)
        
    if algorithm == 'basic':
        forecast_df = forecast_basic(df, current_date, period_length)
    else:
        raise ValueError(f"El algoritmo '{algorithm}' no está implementado.")

    return forecast_df


### 2) PRUEBA con DATOS

In [15]:
# ELEGIR el PROVEEDOR

proveedor = 20
label = '20_Molinos'

# Se puede leer direto de la BD de SQL o desde un archivo CSV local previamente

# Cargar Datos
data = pd.read_csv(f'data/{label}.csv')
data.head()

# Adecuar Tipos de Datos
data['Sucursal']= data['Sucursal'].astype(int)
data['Familia']= data['Familia'].astype(int)
data['Rubro']= data['Rubro'].astype(int)
data['SubRubro']= data['SubRubro'].astype(int)
data['Clasificacion']= data['Clasificacion'].astype(int)
data['Codigo_Articulo']= data['Codigo_Articulo'].astype(int)
data['Fecha'] = pd.to_datetime(data['Fecha'])  # Convertir a formato datetime si aún no lo está
data.sort_values(by='Fecha', ascending=True)  # Ordenar por fecha de menor a mayor

# Crear una nueva columna con el mes y el año para análisis temporal
data['Año-Mes'] = data['Fecha'].dt.to_period('M')

# Crear una nueva columna con el formato Año-Semana (AAAA-WW) a partir de la columna Fecha
data['Año-Semana'] = data['Fecha'].dt.strftime('%Y-%W')

# Confirmar que la columna sigue siendo un campo datetime
#print(data.dtypes)

#data = data.sort_values(by='Fecha', ascending=True)  # Ordenar en orden ascendente (del más antiguo al más reciente)
#data = data.reset_index()

# Recortar Cantidad de Datos ULTIMO AÑO COMPLETO
df = data[data['Fecha']>='2021-01-01']

In [16]:
# Mostrar los resultados
# import ace_tools_open as tools
tools.display_dataframe_to_user(name="SET de Datos del Proveedor", dataframe=df)

SET de Datos del Proveedor


In [17]:
# Calcular la demanda estimada utilizando el algoritmo básico
forecast_result = get_forecast(df, algorithm='basic',current_date='2024-01-01', period_length=15)

# Mostrar los resultados
# import ace_tools_open as tools
tools.display_dataframe_to_user(name="Demanda estimada utilizando el algoritmo básico:", dataframe=forecast_result)

Demanda estimada utilizando el algoritmo básico:


### 3) BACK TESTING  

* Generamos 4 períodos sobre la base histórica.
* Simulamos la fecha actual dentro del período de Datos.
* Comparamos contra FORECAST contra ACTUAL que sería el Real
* Graficamos


In [18]:
current_date ='2024-01-15'   # Establecemos un Curernt Date dentro del Rango de Datos
period_length=15

current_date =  pd.to_datetime(current_date)

# Definir rangos de fechas para cada período
last_period_start = current_date - pd.Timedelta(days=period_length - 1)
last_period_end = current_date

previous_period_start = current_date - pd.Timedelta(days=2 * period_length - 1)
previous_period_end = current_date - pd.Timedelta(days=period_length)

same_period_last_year_start = current_date - pd.DateOffset(years=1) - pd.Timedelta(days=period_length - 1)
same_period_last_year_end = current_date - pd.DateOffset(years=1)

actual_period_start = current_date + pd.Timedelta(+ 1)
actual_period_end = current_date + pd.Timedelta(days=period_length + 2)

In [19]:
# CALCULAMOS LA DEMANDA y la SIMULACIÓN

# Facctores de Pnderación
# Cada uno de los 3 períodos
factor_last = 70
factor_previous = 20
factor_year = 10

 # Filtrar los datos para cada uno de los períodos
df_last = df[(df['Fecha'] >= last_period_start) & (df['Fecha'] <= last_period_end)]
df_previous = df[(df['Fecha'] >= previous_period_start) & (df['Fecha'] <= previous_period_end)]
df_same_year = df[(df['Fecha'] >= same_period_last_year_start) & (df['Fecha'] <= same_period_last_year_end)]
df_actual = df[(df['Fecha'] >= actual_period_start) & (df['Fecha'] <= actual_period_end)]

# Agregar las ventas (unidades) por artículo y sucursal para cada período
sales_last = df_last.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                  .sum().reset_index().rename(columns={'Unidades': 'ventas_last'})
sales_previous = df_previous.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                           .sum().reset_index().rename(columns={'Unidades': 'ventas_previous'})
sales_same_year = df_same_year.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                              .sum().reset_index().rename(columns={'Unidades': 'ventas_same_year'})
sales_actual = df_actual.groupby(['Codigo_Articulo', 'Sucursal'])['Unidades'] \
                           .sum().reset_index().rename(columns={'Unidades': 'ventas_actual'})

# Unir la información de los tres períodos
forecast_df = pd.merge(sales_last, sales_previous, on=['Codigo_Articulo', 'Sucursal'], how='outer')
forecast_df = pd.merge(forecast_df, sales_same_year, on=['Codigo_Articulo', 'Sucursal'], how='outer')
forecast_df = pd.merge(forecast_df, sales_actual, on=['Codigo_Articulo', 'Sucursal'], how='outer')
forecast_df.fillna(0, inplace=True)

# Calcular la demanda estimada como el promedio de las ventas de los tres períodos
forecast_df['forecast'] = (forecast_df['ventas_last'] * factor_last +
                            forecast_df['ventas_previous'] * factor_previous +
                            forecast_df['ventas_same_year'] * factor_year) / (factor_year + factor_last + factor_previous)

# Redondear la predicción al entero más cercano
forecast_df['forecast'] = forecast_df['forecast'].round().astype(int)

In [9]:
# Mostrar resultado
#import ace_tools_open  as tools
tools.display_dataframe_to_user(name="DataFrame Básico", dataframe=forecast_df)

DataFrame Básico


### METRICAS DEL MODELO

In [20]:
# Calcular las métricas de error
mae = np.mean(np.abs(forecast_df['forecast'] - forecast_df['ventas_actual']))
mse = np.mean((forecast_df['forecast'] - forecast_df['ventas_actual'])**2)
# Se añade un valor pequeño para evitar división por cero
mape = np.mean(np.abs((forecast_df['forecast'] - forecast_df['ventas_actual']) / (forecast_df['ventas_actual'] + 1e-9))) * 100

print("Métricas de evaluación:\n",  {'MAE': mae, 'MSE': mse, 'MAPE': mape})

Métricas de evaluación:
 {'MAE': 55.99295097725088, 'MSE': 20278.351169496957, 'MAPE': 931752643539.7194}


#### 1. MAE (Mean Absolute Error - Error Absoluto Medio)

* **Interpretación:**
Representa el promedio de los errores absolutos entre las predicciones y los valores reales.
Se mide en las mismas unidades que la variable objetivo.
Es fácil de interpretar, ya que indica el error promedio sin considerar la dirección del error (positivo o negativo).
* **Ejemplo:**
Si el MAE es 5, significa que, en promedio, las predicciones del modelo se desvían en **5 unidades** de los valores reales.

#### 2. MSE (Mean Squared Error - Error Cuadrático Medio)

* **Interpretación:**
Representa el promedio de los errores al cuadrado.
Penaliza más los errores grandes que los pequeños, ya que eleva al cuadrado las diferencias entre los valores reales y predichos.
Se mide en las unidades al cuadrado de la variable objetivo, lo que a veces dificulta la interpretación directa.
* **Ejemplo:**
Si el MSE es 25 y la variable objetivo se mide en metros, significa que el error cuadrático medio es de 25 metros cuadrados, lo cual no es intuitivo.
* **Uso común:**
Se prefiere cuando se quiere penalizar más los errores grandes.
Se usa mucho en problemas de optimización de modelos porque es una función diferenciable.

#### 3. MAPE (Mean Absolute Percentage Error - Error Porcentual Absoluto Medio)
* **Interpretación:**

Expresa el error en términos porcentuales con respecto al valor real.
Permite comparar el desempeño de modelos en diferentes escalas de datos.
No se ve afectado por la magnitud de la variable objetivo.
* **Ejemplo:**
Si el MAPE es 10%, significa que, en promedio, las predicciones del modelo tienen un error del 10% con respecto a los valores reales.
* **Limitaciones:**
No es ideal cuando hay valores reales cercanos a cero, ya que puede generar valores muy grandes o incluso errores de división por cero.

In [21]:
# Mostrar los resultados
# import ace_tools_open as tools
tools.display_dataframe_to_user(name="Predicciones de Compra Sin Negativos", dataframe=forecast_result)

Predicciones de Compra Sin Negativos
